# Intro Tutorial on Relation Extraction from News Articles

## Part IV: Training a Model with Data Programming

In this part of the tutorial, we will train a statistical model to differentiate between true and false `Disease` mentions.

We will train this model using _data programming_, and we will **ignore** the training labels provided with the training data. This is a more realistic scenario; in the wild, hand-labeled training data is rare and expensive. Data programming enables us to train a model using only a modest amount of hand-labeled data for validation and testing. For more information on data programming, see the [NIPS 2016 paper](https://arxiv.org/abs/1605.07723).

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import numpy as np
from snorkel import SnorkelSession
session = SnorkelSession()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


We repeat our definition of the `Disease` `Candidate` subclass from Parts II and III.

In [2]:
from snorkel.models import candidate_subclass

Spouse = candidate_subclass('Spouse', ['person1', 'person2'])

## Loading `CandidateSet` objects

We reload the training and development `CandidateSet` objects from the previous parts of the tutorial.

In [3]:
from snorkel.models import CandidateSet

train = session.query(CandidateSet).filter(CandidateSet.name == 'News Training Candidates').one()
dev = session.query(CandidateSet).filter(CandidateSet.name == 'News Development Candidates').one()

## Automatically Creating Features
Recall that our goal is to distinguish between true and false mentions of chemical-disease relations. To train a model for this task, we first embed our `ChemicalDisease` candidates in a feature space.

In [4]:
from snorkel.annotations import FeatureManager

feature_manager = FeatureManager()

We can create a new feature set:

In [5]:
%time F_train = feature_manager.create(session, train, 'Train Features')

[=                                       ] 0%

/Users/bradenhancock/snorkel/snorkel/utils.py:16: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  sys.stdout.write("\r[%s%s] %d%%" % ("="*b, " "*(self.length-b), 100*((i+1) / self.nf)))


[========================================] 100%

Loading sparse Feature matrix...
CPU times: user 12min 36s, sys: 4.91 s, total: 12min 41s
Wall time: 12min 45s


**OR** if we've already created one, we can simply load as follows:

In [ ]:
%time F_train = feature_manager.load(session, train, 'Train Features')

Note that the returned matrix is a special subclass of the `scipy.sparse.csr_matrix` class, with some special features which we demonstrate below:

In [6]:
F_train

<1160x22409 sparse matrix of type '<type 'numpy.float64'>'
	with 117845 stored elements in Compressed Sparse Row format>

In [7]:
F_train.get_candidate(0)

Spouse(Span("Ho Ching", parent=593, chars=[44,51], words=[8,9]), Span("Lee Hsien Loong", parent=593, chars=[186,200], words=[37,39]))

In [8]:
F_train.get_key(0)

AnnotationKey (TDL_LEMMA:PARENTS-OF-BETWEEN-MENTION-and-MENTION[None])

## Creating Labeling Functions
Labeling functions are a core tool of data programming. They are heuristic functions that aim to classify candidates correctly. Their outputs will be automatically combined and denoised to estimate the probabilities of training labels for the training data.

In [ ]:
import re
from snorkel.lf_helpers import get_left_tokens, get_right_tokens

## Applying Labeling Functions

First we construct a `CandidateLabeler`.

In [ ]:
from snorkel.annotations import LabelManager

label_manager = LabelManager()

Next we run the `CandidateLabeler` to to apply the labeling functions to the training `CandidateSet`.  We'll start with some of our labeling functions:

In [ ]:
def LF_too_far_apart(c):
    if len(get_text_between(c)) > 10:
        return -1

def LF_third_wheel(c):
    if 'PERSON' in 

    

Candidates with person mentions that are too far apart in the sentence are marked as false.
Candidates with person mentions that have another person in between are marked as false.
Candidates with person mentions that have words like "wife" or "husband" in between are marked as true.
Candidates with person mentions that have "and" in between and "married" after are marked as true.
Candidates with person mentions that have familial relation words in between are marked as false.

In [ ]:
LFs = LFs_mention + LFs_dicts + LFs_false
%time L_train = label_manager.create(session, train, 'LF Labels', f=LFs)
L_train

**OR** load if we've already created:

In [ ]:
%time L_train = label_manager.load(session, train, 'LF Labels')
L_train

We can also add or rerun a single labeling function (or more!) with the below command. Note that we set the argument `expand_key_set` to `True` to indicate that the set of matrix columns should be allowed to expand:

In [ ]:
LFs_2   = LFs_doc + LFs_sent
L_train = label_manager.update(session, train, 'LF Labels', True, f=LFs_2)
L_train

We can view statistics about the resulting label matrix:

In [ ]:
L_train.lf_stats()

## Fitting the Generative Model
We estimate the accuracies of the labeling functions without supervision. Specifically, we estimate the parameters of a `NaiveBayes` generative model.

In [ ]:
from snorkel.learning import NaiveBayes

gen_model = NaiveBayes()
gen_model.train(L_train, n_iter=3000, rate=1e-5)

In [ ]:
gen_model.save(session, 'Generative Params')

We now apply the generative model to the training candidates.

In [ ]:
train_marginals = gen_model.marginals(L_train)

## Training the Discriminative Model
We use the estimated probabilites to train a discriminative model that classifies each `Candidate` as a true or false mention.

In [ ]:
from snorkel.learning import LogReg

disc_model = LogReg()
disc_model.train(F_train, train_marginals, n_iter=5000, rate=1e-3)

In [ ]:
disc_model.w.shape

In [ ]:
%time disc_model.save(session, "Discriminative Params")

## Evaluating on the Development `CandidateSet`

First, we create features for the development set.

Note that we use the training features feature set, because those are the only features for which we have learned parameters. Features that were not encountered during training, e.g., a token that does not appear in the training set, are ignored, because we do not have any information about them.

To do so with the `FeatureManager`, we call update with the new `CandidateSet`, the name of the training `AnnotationKeySet`, and the value `False` for the parameter `extend_key_set` to indicate that the `AnnotationKeySet` should not be expanded with new `Feature` keys encountered during processing.

In [ ]:
%time F_dev = feature_manager.update(session, dev, 'Train Features', False)

**OR** if we've already created one, we can simply load as follows:

In [ ]:
%time F_dev = feature_manager.load(session, dev, 'Train Features')

Next, we load the development set labels and gold candidates we made in Part III.

In [ ]:
L_dev = label_manager.load(session, dev, "News Development Labels -- Gold")

In [ ]:
gold_dev_set = session.query(CandidateSet).filter(CandidateSet.name == 'News Development Candidates').one()

Now we can evaluate the discriminative model on the development set.

In [ ]:
tp, fp, tn, fn = disc_model.score(F_dev, L_dev, gold_dev_set)

## Viewing Examples
After evaluating on the development `CandidateSet`, the labeling functions can be modified. Try changing the labeling functions to improve performance. You can view the true positives, false positives, true negatives, and false negatives using the `Viewer`.

In [ ]:
from snorkel.viewer import SentenceNgramViewer

# NOTE: This if-then statement is only to avoid opening the viewer during automated testing of this notebook
# You should ignore this!
import os
if 'CI' not in os.environ:
    sv = SentenceNgramViewer(tp, session, annotator_name="Tutorial Part IV User")
else:
    sv = None

In [ ]:
sv

Next, in Part V, we will test our model on the test `CandidateSet`.